<a id="1"></a> <br>

# **What Is Bayesian Optimization and Why Do We Care?**

**Bayesian Optimization** is a probabilistic model based approach for finding the minimum of any function that returns a real-value metric. [(source)](https://towardsdatascience.com/an-introductory-example-of-bayesian-optimization-in-python-with-hyperopt-aae40fff4ff0)<br> It is very effective with real-world applications in high-dimensional parameter-tuning for complex machine learning algorithms. Bayesian optimization utilizes the Bayesian technique of setting a prior over the objective function and
combining it with evidence to get a posterior function. I attached one graph that demonstrates Bayes’ theorem below. <br> <br>

<img src="https://www.analyticsvidhya.com/wp-content/uploads/2016/06/12-768x475.jpg"  alt="Drawing" style="width: 600px;"/>
<br> <br> 
 The prior belief is our belief in parameters before modeling process. The posterior belief is our belief in our parameters after observing the evidence.
<br> Another way to present the Bayes’ theorem is: 

<img src="https://www.maths.ox.ac.uk/system/files/attachments/Bayes_0.png"  alt="Drawing" style="width: 600px;"/> <br> 

For continuous functions, Bayesian optimization typically works by assuming the unknown function was sampled from a Gaussian process and maintains a posterior distribution for this function as observations are made, which means that we need to give range of values of hyperparameters (ex. learning rate range from 0.1 to 1).  So, in our case, the Gaussian process gives us a prior distribution on functions. Gaussian process approach is a non-parametric approach, in that it finds a distribution over the possible functions 
f(x) that are consistent with the observed data. Gaussian processes have proven to be useful surrogate models for computer experiments and good
practices have been established in this context for sensitivity analysis, calibration and prediction While these strategies are not considered in the context of optimization, they can be useful to researchers in machine learning who wish to understand better the sensitivity of their models to various hyperparameters. [(source)](http://papers.nips.cc/paper/4522-practical-bayesian-optimization-of-machine-learning-algorithms.pdf)

<br>
**Well..why do we care about it?** <br> 
According to the [study](http://proceedings.mlr.press/v28/bergstra13.pdf), hyperparameter tuning by Bayesian Optimization of machine learnin models is more efficient than Grid Search and Random Search. Bayesian Optimization has better overall performance on the test data and takes less time for optimization. Also, we do not need to set a certain values of parameters like we do in Random Search and Grid Search. For Bayesian Optimization tuning, we just give a range of a hyperparameter. 



<a id="2"></a> <br>
# **Loading Library and Dataset**


In [1]:
#basic tools 
import os
import numpy as np
import pandas as pd
import warnings

#tuning hyperparameters
from bayes_opt import BayesianOptimization
from skopt  import BayesSearchCV 

#graph, plots
import matplotlib.pyplot as plt
import seaborn as sns

#building models
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import time
import sys

#metrics 
from sklearn.metrics import roc_auc_score, roc_curve
import shap
warnings.simplefilter(action='ignore', category=FutureWarning)

By Changing the data type of each column, I reduced memory usages by 75%. By taking the minimum and the maximum of each column, the function assigns which numeric data type is optimal for the column and change the data type. If you want to know more about how it works, I suggest you to read [Eryk's article](https://towardsdatascience.com/make-working-with-large-dataframes-easier-at-least-for-your-memory-6f52b5f4b5c4)! 

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
%%time
train= reduce_mem_usage(pd.read_csv("../input/train.csv"))
test= reduce_mem_usage(pd.read_csv("../input/test.csv"))
print("Shape of train set: ",train.shape)
print("Shape of test set: ",test.shape)

Mem. usage decreased to 78.01 Mb (74.7% reduction)
Mem. usage decreased to 77.82 Mb (74.6% reduction)
Shape of train set:  (200000, 202)
Shape of test set:  (200000, 201)
CPU times: user 31.3 s, sys: 31.8 s, total: 1min 3s
Wall time: 1min 3s
